In [20]:
#import astroprov

from astropy.io import fits
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table
from astropy.table import Column
from astropy.io import fits
from astropy.io.ascii import SExtractor
from astropy.io import ascii
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy import wcs

from astroquery.simbad import Simbad

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
# from photutils import DAOStarFinder
# from photutils import CircularAperture
import os
import subprocess

import image_registration
from image_registration import chi2_shift
from image_registration.fft_tools import shift
import pandas
import collections

#from reproject import reproject_interp
from astropy.utils.data import get_pkg_data_filename
from astropy.wcs import WCS

from scipy.stats import iqr

In [87]:
imageDirectory = '/media/mj1e16/PP AV-TV/keplerCal/'
#medianDir = '/data/mj1e16/kepler/newDiff/'

dirlist = os.listdir(imageDirectory)
dirlist = [x for x in dirlist if x[0] != '.']
imagelist = [imageDirectory+x for x in dirlist]

difDir = '/media/mj1e16/PP AV-TV/properDiff/'
diffdirlist = os.listdir(difDir)
diffdirlist = [x for x in diffdirlist if x[0] != '.']
print(len(diffdirlist))
diffdirlist = [x for x in diffdirlist if 'fits' in x]
print(len(diffdirlist))
diffimagelist = [difDir+x for x in diffdirlist]

4453
4452


In [113]:
testIm = ['/media/mj1e16/PP AV-TV/properDiff/diff_imNo0_channel2_ccd54.fits']
chanAndCcd = [im[im.index('channel'):im[im.index('channel'):].index('_')+im.index('channel')] + im[im.index('ccd'):im[im.index('ccd'):].index('.fits')+im.index('ccd')] for im in testIm]
print(chanAndCcd)

['channel2ccd54']


In [111]:
testIm = ['/media/mj1e16/PP AV-TV/properDiff/diff_imNo0_channel2_ccd54.fits']
print(testIm[0][testIm[0].index('channel'):(im[im.index('channel'):].index('_'))+im.index('channel')])
chanAndCcd = [im[(im.index('channel')):(im[im.index('channel'):].index('_'))+im.index('channel')] for im in testIm]
print(chanAndCcd)

channel2
['channel2']


In [88]:
print(diffimagelist[0])

/media/mj1e16/PP AV-TV/properDiff/diff_imNo0_channel2_ccd54.fits


In [91]:
difGrouping = {}

for im in range(len(diffimagelist)):
    loc1 = diffimagelist[im].index('imNo') 
    loc2 = diffimagelist[im][loc1:].index('_') + loc1
    imNo = diffimagelist[im][loc1:loc2]
    
    loc3 = diffimagelist[im].index('channel')
    loc4 = diffimagelist[im][loc3:].index('_') + loc3
    channel = diffimagelist[im][loc3:loc4]
    
    loc5 = diffimagelist[im].index('ccd')
    loc6 = diffimagelist[im][loc5:].index('.') + loc5
    ccd = diffimagelist[im][loc5:loc6]
    
    if channel+ccd in difGrouping:
        difGrouping[channel+ccd].append(diffimagelist[im])
    else:
        difGrouping[channel+ccd] = [diffimagelist[im]]
    
    

In [96]:
(84*53)/13

342

In [94]:
len(difGrouping)

336

In [3]:
def getSkyGroup(imagelist,image,channel):
    hdu_list = fits.open(imagelist[image])
    hdr = hdu_list[channel].header
    location = hdr['SKYGROUP']
    return location

In [4]:
def getImageData(imageName,extension):
    hdu = fits.open(imageName)
    imageData = hdu[extension].data
    return imageData

In [5]:
def correctExtensionOrder(imageName):
    hdu_list = fits.open(imageName)
    extensionOrder = np.zeros(len(hdu_list))
    for channel in range(1,len(hdu_list)):
        hdr = hdu_list[channel].header
        location = hdr['SKYGROUP'] # independant location
        extensionOrder[location] = channel # ordered from 0-84 channel with extension in place of name
    return extensionOrder

In [6]:
extensionOrders = []
for image in imagelist:
    extensionOrders.append(correctExtensionOrder(image))

In [7]:
imagelist[1]

'/media/mj1e16/PP AV-TV/keplerCal/kplr2009114204835_ffi-cal.fits'

In [8]:
imageBunching = [[],[],[],[]]
extensions = []
for image in range(len(imagelist)):
    elist = extensionOrders[image].tolist()
    if elist not in extensions:
        extensions.append(elist)
        imageBunching[len(extensions)-1].append(imagelist[image])
    else:
        loc = extensions.index(elist)
        imageBunching[loc].append(imagelist[image])

In [9]:
#imageBunching

In [10]:
iqrFull = []
for imGroup in range(len(imageBunching)):
    imLoc = imagelist.index(imageBunching[imGroup][0])
    channels = extensionOrders[imLoc]
    iqrPerGroup = []
    for chan in range(1,len(channels)):
        imDataList = []
        for im in range(len(imageBunching[imGroup])):
            imDataList.append(getImageData(imageBunching[imGroup][im],int(channels[chan])))
        arrayStack = np.stack(imDataList)
        std = np.std(arrayStack,axis=0)
        interQRange = iqr(std)
        iqrPerGroup.append(interQRange)
    iqrFull.append(iqrPerGroup)
#     for im in range(len(imageBunching[imGroup])):
#         imLoc = 
#         for chan in range(len(extensionOrders[imLoc])):
            

In [97]:
iqrGroupedbyChannel = {}
for key,value in difGrouping.items():
    imDataList = []
    for im in range(len(value)):
        imDataList.append(getImageData(value[im],0))
    arrayStack = np.stack(imDataList)
    std = np.std(arrayStack,axis=0)
    interQRange = iqr(std)
    iqrGroupedbyChannel[key] = interQRange

In [38]:
imNametoGroup = {}
for groupNum in range(len(imageBunching)):
    for im in imageBunching[groupNum]:
        imNametoGroup[im] = groupNum
#imageBunching

In [12]:
std = np.std(arrayStack,axis=0)

In [13]:
interQRange = iqr(std)

In [78]:
len(iqrFull[0])

84

In [69]:
# gain = 110

# for group in range(len(iqrFull)):
#     iqrFull[group] = [x/gain for x in iqrFull[group]]

In [21]:
firstTab = ascii.read('/home/mj1e16/outTablesImproved/final/firsthalf.csv',format='csv')
#secondTab = ascii.read('/home/mj1e16/outTablesImproved/final/',format='csv')

In [46]:
imageNames = firstTab['IMAGE_NAME'].tolist()

In [60]:
for x in range(len(imageNames)):
    if type(imageNames[x]) != str:
        print(imageNames[x])
        print(type(imageNames[x]))
        print(x)
firstTab.remove_row(x)

None
<type 'NoneType'>
810804


In [ ]:
    loc3 = diffimagelist[im].index('channel')
    loc4 = diffimagelist[im][loc3:].index('_') + loc3
    channel = diffimagelist[im][loc3:loc4]
    
    loc5 = diffimagelist[im].index('ccd')
    loc6 = diffimagelist[im][loc5:].index('.') + loc5
    ccd = diffimagelist[im][loc5:loc6]

In [115]:
imageNames = firstTab['DIFF_NAME'].tolist()
chanAndCcd = [im[im.index('channel'):im[im.index('channel'):].index('_')+im.index('channel')] + im[im.index('ccd'):im[im.index('ccd'):].index('.fits')+im.index('ccd')] for im in imageNames]
# imNameOnly = [x[:x.index('[')] for x in imageNames]
# ccdOnly = [int(x[x.index('[')+1:x.index(']')]) for x in imageNames]
fluxBest = firstTab['FLUX_BEST'].tolist()
Noise = [iqrGroupedbyChannel[im] for im in chanAndCcd]
SNR = [f/N for f,N in zip(fluxBest,Noise)]


In [126]:
firstTab =  ascii.read('/home/mj1e16/outTablesImproved/final/firsthalf.csv',format='csv')
SNRbelow10 = [i for i,j in enumerate(SNR)  if j <10]
firstTab.remove_rows(SNRbelow10)
firstTab.write('/home/mj1e16/outTablesImproved/newSNR/10snrfirst.csv',format='csv')

firstTab =  ascii.read('/home/mj1e16/outTablesImproved/final/firsthalf.csv',format='csv')
SNRbelow100 = [i for i,j in enumerate(SNR)  if j <100]
firstTab.remove_rows(SNRbelow100)
firstTab.write('/home/mj1e16/outTablesImproved/newSNR/100snrfirst.csv',format='csv')

firstTab =  ascii.read('/home/mj1e16/outTablesImproved/final/firsthalf.csv',format='csv')
SNRbelow1000 = [i for i,j in enumerate(SNR)  if j <1000]
firstTab.remove_rows(SNRbelow1000)
firstTab.write('/home/mj1e16/outTablesImproved/newSNR/1000snrfirst.csv',format='csv')

firstTab =  ascii.read('/home/mj1e16/outTablesImproved/final/firsthalf.csv',format='csv')
SNRbelow10000= [i for i,j in enumerate(SNR)  if j <10000]
firstTab.remove_rows(SNRbelow10000)
firstTab.write('/home/mj1e16/outTablesImproved/newSNR/10000snrfirst.csv',format='csv')



In [127]:
len(firstTab)

1563

In [59]:
firstTab[810804

NUMBER,FLUX_BEST,FLUXERR_BEST,MAG_BEST,MAGERR_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY,RA,DEC,IMAGE_NAME,MEDIAN_NAME,DIFF_NAME
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str34,str18,str32
159,554.2703,15.83873,18.1407,0.031,1095.3187,729.3903,--,--,--,--,--,--


In [56]:
if type(imageNames[0]) == str:
    print('is string')

is string


In [47]:


ccdOnly = [x[x.index('[')+1:x.index(']')] for x in imageNames]
for x in range(len(ccdOnly)):
    print(ccdOnly[x])
    

AttributeError: 'NoneType' object has no attribute 'index'

In [44]:
print(y)

NameError: name 'y' is not defined

In [66]:
imNametoGroup['/media/mj1e16/PP AV-TV/keplerCal/kplr2009351005245_ffi-cal.fits']

2

In [67]:
imageNames = firstTab['IMAGE_NAME'].tolist()
imNameOnly = [x[:x.index('[')] for x in imageNames]
ccdOnly = [int(x[x.index('[')+1:x.index(']')]) for x in imageNames]
fluxBest = firstTab['FLUX_BEST'].tolist()
Noise = [iqrFull[imNametoGroup[imageDirectory+im]][ccd-1] for im,ccd in zip(imNameOnly,ccdOnly)]
SNR = [f/N for f,N in zip(fluxBest,Noise)]

In [68]:
SNR[0:10]

[16.131561356247975,
 2.9781256520982637,
 23.635534008842111,
 1.0099208031082636,
 3.7281403493269996,
 2.1660513187313528,
 3.0214883256992247,
 2.2634958250762653,
 3.4436998731048365,
 3.3301065669340297]

In [34]:
imageNames[0][:imageNames[0].index('[')]

'kplr2012310200152_ffi-cal.fits'

In [27]:
firstTab[0:10]

NUMBER,FLUX_BEST,FLUXERR_BEST,MAG_BEST,MAGERR_BEST,X_IMAGE,Y_IMAGE,ELLIPTICITY,RA,DEC,IMAGE_NAME,MEDIAN_NAME,DIFF_NAME
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str34,str18,str32
1,4225.085,28.74944,15.9354,0.0074,958.0409,25.5088,0.697,288.882155336,41.6027532272,kplr2012310200152_ffi-cal.fits[58],meidian_66_58.fits,diff_imNo23_channel66_ccd58.fits
2,780.0134,16.32858,17.7697,0.0227,965.0,24.4539,0.874,288.872777757,41.5993971256,kplr2012310200152_ffi-cal.fits[58],meidian_66_58.fits,diff_imNo23_channel66_ccd58.fits
3,6190.482,79.87542,15.5207,0.014,958.0,43.9386,0.79,288.897504363,41.5859447684,kplr2012310200152_ffi-cal.fits[58],meidian_66_58.fits,diff_imNo23_channel66_ccd58.fits
4,264.5126,31.40866,18.9439,0.129,318.3539,29.9193,0.644,289.672185706,41.9930764063,kplr2012310200152_ffi-cal.fits[58],meidian_66_58.fits,diff_imNo23_channel66_ccd58.fits
5,976.4529,18.9721,17.5259,0.0211,1014.9861,20.2416,0.255,288.808245766,41.5721972161,kplr2012310200152_ffi-cal.fits[58],meidian_66_58.fits,diff_imNo23_channel66_ccd58.fits
6,567.3196,14.60473,18.1154,0.028,553.0087,19.6884,0.17,289.37413918,41.858419875,kplr2012310200152_ffi-cal.fits[58],meidian_66_58.fits,diff_imNo23_channel66_ccd58.fits
7,791.3707,18.9721,17.754,0.026,325.0489,26.2907,0.348,289.660914047,41.992311294,kplr2012310200152_ffi-cal.fits[58],meidian_66_58.fits,diff_imNo23_channel66_ccd58.fits
8,592.8417,17.51046,18.0677,0.0321,948.2596,29.3369,0.566,288.897284722,41.6053266953,kplr2012310200152_ffi-cal.fits[58],meidian_66_58.fits,diff_imNo23_channel66_ccd58.fits
9,901.9539,25.55828,17.612,0.0308,999.3637,36.2413,0.224,288.840604615,41.5672961624,kplr2012310200152_ffi-cal.fits[58],meidian_66_58.fits,diff_imNo23_channel66_ccd58.fits
